## Import

In [12]:
import pandas as pd
import numpy as np

## Exploring data

### Read raw data from file

In [13]:
weather_df = None
weather_df = pd.read_csv('../data/internal/raw/weather.csv')

In [14]:
# weather_df = weather_df.drop("Rain", axis='columns')

In [15]:
weather_df.head()

,Country,Name,Weather,Temp,Humidity,Visibility,Wind speed,Clouds
0,JP,Tokyo,Clear,280.24,58,10000,3.60,0
1,ID,Pecenongan,Haze,299.25,86,6000,2.06,20
2,IN,New Delhi,Mist,292.25,88,1000,0.00,0
3,CN,Guangzhou,Clouds,288.07,47,10000,4.62,24
4,IN,Konkan Division,Mist,300.15,83,2200,4.12,20


### How many rows and how many columns does the raw data have?

In [16]:
shape = None 
shape = weather_df.shape
print(shape)

(3639, 8)


### What does each line mean?


Each line contains information about the weather of a city.

### Does the raw data have duplicate rows?

In [17]:
num_duplicated_rows = None
num_duplicated_rows = weather_df.duplicated().sum()

In [18]:
#TEST
if num_duplicated_rows == 0:
    print(f"The raw data have no duplicated line!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"The raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate the raw data!")

The raw data have no duplicated line!


In [19]:
#De-deduplicate the raw data
weather_df = weather_df.drop_duplicates()

In [20]:
#TEST
num_duplicated_rows = weather_df.duplicated().sum()
if num_duplicated_rows == 0:
    print(f"The raw data have no duplicated line!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"The raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data!")

The raw data have no duplicated line!


### What does each column mean?

- Country: Country ID
- Name: City's name
- Weather: Weather situation
- Temp: Current temperature
- Humidity: Current humidity
- Visibility: Current vision
- Wind speed: Current wind speed
- Clouds: Cloud frequency

### What data type does each column currently have? Are there any columns whose data types are not suitable for further processing?

In [21]:
dtypes = None
dtypes = weather_df.dtypes

In [22]:
dtypes

Country        object
Name           object
Weather        object
Temp          float64
Humidity        int64
Visibility      int64
Wind speed    float64
Clouds          int64
dtype: object

There are not any columns whose data types are not suitable for further processing

### For each column with numeric data type, how are the values distributed?

In [23]:
num_col_info_df = None

num_col_info_df = weather_df.select_dtypes(include=np.number).copy()

def missing_ratio(column):
    return ((column.isnull().sum() / column.shape[0]) * 100).round(1)

def lower_quartile(column):
    return (column.quantile(0.25)).round(1)

def median(column):
    return (column.median())

def upper_quartile(column):
    return (column.quantile(0.75)).round(1)

# Làm tròn giá trị đến 1 chữ số thập phân
num_col_info_df = num_col_info_df.round(1)

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

In [24]:
num_col_info_df

,Temp,Humidity,Visibility,Wind speed,Clouds
missing_ratio,0.0,0.0,0.0,0.0,0.0
min,247.1,9.0,33.0,0.0,0.0
lower_quartile,278.3,51.0,10000.0,1.5,5.0
median,289.6,69.0,10000.0,2.5,41.0
upper_quartile,298.1,85.0,10000.0,3.8,92.0
max,313.8,100.0,10000.0,16.1,100.0


### Drop the column of which the percentage of missing values is greater than 75%

In [25]:
def drop_missing_features(df: pd.DataFrame, missing_lst: dict = dict(num_col_info_df.iloc[0]), threshold: float = 75.0) -> pd.DataFrame:
    for col, missing_rate in missing_lst.items():
        if missing_rate > threshold:
            del df[col]
    return df

In [26]:
weather_df = drop_missing_features(weather_df)

In [27]:
weather_df.head()

,Country,Name,Weather,Temp,Humidity,Visibility,Wind speed,Clouds
0,JP,Tokyo,Clear,280.24,58,10000,3.60,0
1,ID,Pecenongan,Haze,299.25,86,6000,2.06,20
2,IN,New Delhi,Mist,292.25,88,1000,0.00,0
3,CN,Guangzhou,Clouds,288.07,47,10000,4.62,24
4,IN,Konkan Division,Mist,300.15,83,2200,4.12,20


### For each column with a non-numeric data type, how are the values distributed?

In [28]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

In [29]:
cat_col_info_df = weather_df.select_dtypes(exclude=[np.number])

def missing_ratio(column):
    return ((column.isnull().sum() / column.shape[0]) * 100).round(1)

# Hàm tính số lượng giá trị
def num_values(column):
    return column.nunique()

# Hàm tính tỷ lệ của từng giá trị
def value_ratios(column):
    value_counts = column.value_counts() #Đếm số lượng của mỗi loại value trong 1 cột
    non_missing_count = value_counts.sum() #Tổng số lượng của tất cả value trong 1 cột
    ratios = (value_counts / non_missing_count * 100).round(1) #Lưu tỉ lệ vào Series
    ratios_dict = ratios.to_dict()
    sorted_ratios_dict = dict(sorted(ratios_dict.items(), key=lambda item: item[1], reverse=True))
    return sorted_ratios_dict

cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,Country,Name,Weather
missing_ratio,0.1,0.1,0.0
num_values,237,3458,10
value_ratios,"{'CN': 17.3, 'IN': 8.5, 'US': 7.3, 'BR': 4.3, 'JP': 4.3, 'RU': 3.0, 'MX': 2.9, 'ID': 2.5, 'PH': ...","{'Minamisenju': 0.1, 'Adana': 0.1, 'Dongguan': 0.1, 'Xinxiang': 0.1, 'Guadalupe': 0.1, 'Depok': ...","{'Clouds': 59.5, 'Clear': 27.4, 'Rain': 5.7, 'Mist': 2.9, 'Snow': 1.9, 'Haze': 1.4, 'Fog': 0.4, ..."


### Save the processed data

In [30]:
print(f"Total number of features: {weather_df.shape[1]}")
weather_df.dtypes

Total number of features: 8


Country        object
Name           object
Weather        object
Temp          float64
Humidity        int64
Visibility      int64
Wind speed    float64
Clouds          int64
dtype: object

In [31]:

weather_df.to_csv("../data/internal/processed/weather_processed.csv" , index=False)